In [ ]:
import yaml

def read_yaml(file_path):
    with open(file_path, 'r') as file:
        try:
            content = yaml.safe_load(file)
            print(content)
        except yaml.YAMLError as exc:
            print(f"Error reading YAML file: {exc}")

# Usage
file_path = 'D:\The Welding Defect Dataset\data.yaml'  # Replace with the path to your YAML file
read_yaml(file_path)

In [ ]:
import ultralytics
ultralytics.checks()

In [ ]:
import os
import torch
from ultralytics import YOLO



# Define the path to your data configuration file
DATA_YAML_PATH = 'D:\The Welding Defect Dataset\data.yaml'

# Select the base model. YOLOv8n (nano) is fast and lightweight, 
# great for initial testing. 
MODEL_NAME = 'yolov8n.pt' 
PROJECT_NAME = 'weld_quality_detection'

# Determine the device dynamically
if torch.cuda.is_available():
    # Use GPU 0 for training/evaluation
    DEVICE = '0'
    print(f"CUDA device detected: Running on GPU {DEVICE}")
else:
    # Use CPU if no CUDA device is found
    DEVICE = 'cpu'
    print("No CUDA device detected. Running on CPU (training may be slow).")


# --- Training Function ---
def train_model():
    
    print(f"\nLoading model: {MODEL_NAME}")
    # Load a pre-trained model
    model = YOLO(MODEL_NAME)
    
    # Check if the data.yaml file exists
    if not os.path.exists(DATA_YAML_PATH):
        print(f"Error: {DATA_YAML_PATH} not found.")
        print("Please ensure 'data.yaml' is in the same directory as this script.")
        return

    print(f"Starting training with data from {DATA_YAML_PATH}...")
    
   
    # This prevents memory fragmentation and ensures all VRAM is available for training.
    if DEVICE != 'cpu':
        torch.cuda.empty_cache()
        print("Cleared CUDA cache to free up VRAM.")
        
    # Train the model
    #  Batch=8, AMP=True, and now ImgSz is reduced to 320.
    results = model.train(
        data=DATA_YAML_PATH, 
        epochs=50,             
        imgsz=320,            
        batch=8,             
        workers=4,             # Reduced data loader workers for stability
        project=PROJECT_NAME,  # Main folder name for all runs
        name='run_1',          # Specific run name
        patience=10,           # Stop training early if no improvement after 10 epochs
        device=DEVICE,         # Use the dynamically determined device (GPU 0)
        amp=True,              # CRITICAL: Enable Automatic Mixed Precision (FP16)
    )

    print("\nTraining complete!")
    print(f"Results are saved to: {PROJECT_NAME}/run_1")
    return model

#Evaluation and Prediction Function
def evaluate_and_predict(model):
    """
    Evaluates the trained model on the validation set and runs prediction on the test set.
    """
    if model is None:
        print("Model not trained or loaded. Skipping evaluation and prediction.")
        return

    # 1. Evaluate the model on the validation set (Val)
    print("\n--- Running Evaluation (Validation Set) ---")
    
    # Clear cache before evaluation as well, just in case.
    if DEVICE != 'cpu':
        torch.cuda.empty_cache()
        
    # Note: Using the trained image size (320) for evaluation is generally best practice.
    metrics = model.val(device=DEVICE, half=True, imgsz=320)

    print(f"\nMean Average Precision (mAP@0.5): {metrics.results_dict['metrics/mAP50(B)']:.4f}")
    print(f"Mean Average Precision (mAP@0.5:0.95): {metrics.results_dict['metrics/mAP50-95(B)']:.4f}")

    # 2. Run prediction on the test set (Test)
    print("\n--- Running Prediction (Test Set) ---")
    
  
    test_results = model.predict(
        source='test/images', 
        save=True,              # Save the resulting images with bounding boxes
        conf=0.25,              # Confidence threshold for detections
        iou=0.7,                # IoU threshold for Non-Maximum Suppression (NMS)
        device=DEVICE,          # Use the dynamically determined device
        half=True,              # Added for memory efficiency during inference
        imgsz=320,              # Added for consistency
    )
    
    print("\nTest predictions saved to the last 'predict' folder in 'runs/detect/'")


# Main Execution
if __name__ == '__main__':
    # Train the model
    trained_model = train_model()
    
    # Evaluate and predict
    evaluate_and_predict(trained_model)

    print("\nProcess finished.")
